In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('afghanForum.csv', error_bad_lines=False)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
messages = df[["Message"]]

In [ ]:
import textblob
import re
import nltk
import string
from collections import Counter

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def word_freq(messages, TEXTS):
    freq_df = messages
    freq_words = freq_df[TEXTS].tolist()
    freq_words = [i.lower() for i in freq_words]
    freq_punc = []
    
    for o in freq_words:
        freq_punc += nltk.word_tokenize(o)
        
    freq_punc = [o for o in freq_punc if o not in string.punctuation]
    freq_freq = Counter(freq_punc)
    
    freq_top = freq_freq.most_common(20)
    words = [word for word, _ in freq_top]
    counts = [counts for _, counts in freq_top]
    
    plt.bar(words, counts)
    plt.title("MOST COMMON 20 WORDS")
    plt.ylabel("Frequency")
    plt.xlabel("Words")
    plt.xticks(rotation=90)
    plt.show()
    
    return freq_top

In [ ]:
freq_top = word_freq(messages, 'Message')

In [ ]:
from wordcloud import WordCloud

In [ ]:
def print_wordcloud(dict_top):
    dict_top = dict(dict_top)
    wordcloud = WordCloud(
        width=500, 
        height=500, 
        background_color='white', 
        min_font_size=5).generate_from_frequencies(dict_top)
    
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

In [ ]:
print_wordcloud(freq_top)

In [ ]:
from nltk.corpus import stopwords
stop_words_list = stopwords.words('english')

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\d', ' ', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [word for word in words if not word in stop_words_list]
    words = [re.sub(r'(.)\1{1,}', r'\1\1', word) for word in words]
    words = [word.strip() for word in words if len(word.strip()) > 1]
    text = ' '.join(words)
    return text

In [ ]:
messages['cleaned'] = messages['Message'].apply(preprocess_text)

In [ ]:
messages.head()

In [ ]:
def get_message_sentiment(message):
    analysis = textblob.TextBlob(message)
    if analysis.sentiment.polarity > 0:
        return 'POSITIVE'
    elif analysis.sentiment.polarity == 0:
        return 'NEUTRAL'
    else:
        return 'NEGATIVE'

In [ ]:
messages["sentiment"] = messages["cleaned"].apply(get_message_sentiment)

In [ ]:
messages.head()

In [ ]:
messages.sentiment.value_counts().plot(kind='pie', autopct='%.2f%%')

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
def label_encoder(column):
    le = LabelEncoder().fit(column)
    print(column.name, le.classes_)
    return le.transform(column)

In [ ]:
messages["sentiment"] = label_encoder(messages["sentiment"])

In [ ]:
messages.head()

In [ ]:
X = messages["cleaned"]
y = messages["sentiment"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("X train: ", X_train.shape)
print("X test: ", X_test.shape)
print("Y train: ", y_train.shape)
print("Y test: ", y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
rf_tfidf = RandomForestClassifier()
rf_tfidf.fit(X_train_tfidf, y_train)
pred_rf_tfidf = rf_tfidf.predict(X_test_tfidf)
ac_rf_tfidf = accuracy_score(pred_rf_tfidf, y_test)
print("RF-TFIDF Accuracy Score: ", ac_rf_tfidf)

In [ ]:
cm = confusion_matrix(pred_rf_tfidf, y_test)
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Pastel1")

In [ ]:
rf_cv = RandomForestClassifier()
rf_cv.fit(X_train_cv, y_train)
pred_rf_cv = rf_cv.predict(X_test_cv)
ac_rf_cv = accuracy_score(pred_rf_cv, y_test)
print("RF-CV Accuracy Score: ", ac_rf_cv)

In [ ]:
cm = confusion_matrix(pred_rf_cv, y_test)
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Pastel1")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_tfidf = DecisionTreeClassifier()
dt_tfidf.fit(X_train_tfidf, y_train)
pred_dt_tfidf = dt_tfidf.predict(X_test_tfidf)
ac_dt_tfidf = accuracy_score(pred_dt_tfidf, y_test)
print("DT-TFIDF Accuracy Score: ", ac_dt_tfidf)

In [ ]:
cm = confusion_matrix(pred_dt_tfidf, y_test)
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Pastel1")

In [ ]:
dt_cv = DecisionTreeClassifier()
dt_cv.fit(X_train_cv, y_train)
pred_dt_cv = dt_cv.predict(X_test_cv)
ac_dt_cv = accuracy_score(pred_dt_cv, y_test)
print("DT-CV Accuracy Score: ", ac_dt_cv)

In [ ]:
cm = confusion_matrix(pred_dt_cv, y_test)
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Pastel1")

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train_tfidf, y_train)
pred_mnb_tfidf = mnb_tfidf.predict(X_test_tfidf)
ac_mnb_tfidf = accuracy_score(pred_mnb_tfidf, y_test)
print("MNB-TFIDF Accuracy Score: ", ac_mnb_tfidf)

In [ ]:
mnb_cv = MultinomialNB()
mnb_cv.fit(X_train_cv, y_train)
pred_mnb_cv = mnb_cv.predict(X_test_cv)
ac_mnb_cv = accuracy_score(pred_mnb_cv, y_test)
print("MNB-CV Accuracy Score: ", ac_mnb_cv)

In [ ]:
models_and_scores = {
    "RF-TFIDF": ac_rf_tfidf,
    "RF-CV": ac_rf_cv,
    "DT-TFIDF": ac_dt_tfidf,
    "DT-CV": ac_dt_cv,
    "MNB-TFIDF": ac_mnb_tfidf,
    "MNB-CV": ac_mnb_cv
}

In [ ]:
plt.bar(*zip(*models_and_scores.items()))